# CLIP推理（效果不好）

In [10]:
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("/home/lyz/hf-models/openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("/home/lyz/hf-models/openai/clip-vit-base-patch16")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities

tensor([[9.9916e-01, 8.4369e-04]], grad_fn=<SoftmaxBackward0>)

In [11]:
import pandas as pd
test = pd.read_csv("re_id_1000_test.txt", header=None)

In [12]:
class_names = ["Toyota_Camry", "Toyota_Corolla", "Toyota_Corolla_EX", "Buick_LaCrosse", "Volkswagen_Magotan", "Audi_A4",
               "Nissan_Sylphy", "Nissan_Tiida", "Honda_Accord", "Ford_Focus"]

In [22]:
image = Image.open("image/" + test[0].values[100].replace("\\", "/"))
inputs = processor(text=["a photo of a " + x for x in class_names], images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)
logits_per_image = outputs.logits_per_image
logits_per_image.softmax(dim=1)

tensor([[0.1830, 0.1186, 0.1731, 0.0035, 0.0034, 0.0037, 0.3545, 0.1169, 0.0231,
         0.0201]], grad_fn=<SoftmaxBackward0>)

# CNN

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import models, transforms, datasets
from transformers import TrainingArguments, Trainer
from datasets import load_metric # Hugging Face utility for metrics
import pandas as pd
from PIL import Image
import numpy as np
import warnings
warnings.filterwarnings("ignore")

2025-11-18 10:48:18.984737: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [49]:
import pandas as pd
test = pd.read_csv("re_id_1000_test.txt", header=None)
train = pd.read_csv("re_id_1000_train.txt", header=None)

test["label"] = test[0].apply(lambda x: int(x.split("\\")[0]) - 1)
test["path"] = test[0].apply(lambda x: "image/" + x.replace("\\", "/"))

train["label"] = train[0].apply(lambda x: int(x.split("\\")[0]) - 1)
train["path"] = train[0].apply(lambda x: "image/" + x.replace("\\", "/"))

In [50]:
test.dtypes

0        object
label     int64
path     object
dtype: object

In [51]:
train.head(3)

0  label  \
0  1\License_1\4404000000002940408492.jpg      0   
1  1\License_1\4404000000002949435646.jpg      0   
2  1\License_1\4404000000002949812715.jpg      0   

                                           path  
0  image/1/License_1/4404000000002940408492.jpg  
1  image/1/License_1/4404000000002949435646.jpg  
2  image/1/License_1/4404000000002949812715.jpg

In [52]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, unique_labels, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.unique_labels = unique_labels
        self.label_to_index = {label: i for i, label in enumerate(self.unique_labels)}
        self.num_classes = len(self.label_to_index)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['path']
        try:
            image = Image.open(img_path).convert('RGB')
        except FileNotFoundError:
            print(f"Error: Image not found at {img_path}. Returning a dummy sample.")
            return torch.zeros(3, 224, 224), 0

        label_name = self.dataframe.iloc[idx]['label']
        label_index = self.label_to_index[label_name]

        if self.transform:
            image = self.transform(image)

        return {'pixel_values': image, 'labels': label_index}

In [53]:
def get_data_transforms():
    return transforms.Compose([
        transforms.Resize(256),         # Resize shortest side to 256
        transforms.CenterCrop(224),     # Center crop to 224x224 (ResNet input size)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    pixel_values = torch.stack([item['pixel_values'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch], dtype=torch.long)
    
    return {'pixel_values': pixel_values, 'labels': labels}

class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, pixel_values, labels=None):
        logits = self.model(pixel_values)
        if labels is not None:
            loss = self.criterion(logits, labels)
            return (loss, logits)
        
        return logits

accuracy_metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

In [63]:
batch_size = 64

transform = get_data_transforms()

train_set = CustomImageDataset(train, train["label"].unique(), transform)
test_set = CustomImageDataset(test, train["label"].unique(), transform)

train_len = int(0.9 * len(train_set))
val_len = len(train_set) - train_len
train_set, val_set = random_split(train_set, [train_len, val_len])

num_classes = 10 
model = ResNetClassifier(num_classes=num_classes)

In [64]:
training_args = TrainingArguments(
    output_dir='./results',                          # Output directory for checkpoints and logs
    num_train_epochs=1,                              # Total number of training epochs
    per_device_train_batch_size=batch_size,          # Batch size per device during training
    per_device_eval_batch_size=batch_size,           # Batch size for evaluation
    warmup_steps=500,                                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                               # Strength of weight decay
    logging_dir='./logs',                            # Directory for storing logs
    logging_steps=50,
    evaluation_strategy="epoch",                     # Evaluate after each epoch
    save_strategy="epoch",                           # Save checkpoint after each epoch
    load_best_model_at_end=True,                     # Load the best model at the end
    metric_for_best_model="accuracy",                # Use accuracy to determine best model
    learning_rate=5e-4,                              # Starting learning rate
    remove_unused_columns=False,                     # Keep all columns in the dataset for flexibility
    dataloader_num_workers=4,
)

trainer = Trainer(
    model=model,                                     # The instantiated model
    args=training_args,                              # Training arguments
    train_dataset=train_set,                         # Training dataset
    eval_dataset=val_set,                            # Validation dataset
    compute_metrics=compute_metrics,                 # Function to compute evaluation metrics
    data_collator=custom_collate_fn,
)

trainer.train()

TrainOutput(global_step=36, training_loss=1.7459883160061307, metrics={'train_runtime': 13.8711, 'train_samples_per_second': 324.415, 'train_steps_per_second': 2.595, 'total_flos': 0.0, 'train_loss': 1.7459883160061307, 'epoch': 1.0})

In [65]:
test_results = trainer.evaluate(test_set)
test_results

{'eval_loss': 0.574944794178009,
 'eval_accuracy': 0.9326,
 'eval_runtime': 11.2664,
 'eval_samples_per_second': 443.799,
 'eval_steps_per_second': 3.55,
 'epoch': 1.0}

In [79]:
def predict_single_image(model, image_path, transform, label_names):
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0) 

        model.eval()
        device = next(model.parameters()).device
        input_batch = input_batch.to(device)

        with torch.no_grad():
            output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output, dim=1)
        probabilities = probabilities[0]
        predicted_index = torch.argmax(probabilities).item()
        predicted_label = label_names[predicted_index]
        confidence = probabilities[predicted_index].item() * 100

        print(f"Predicted Class Index: {predicted_index}")
        print(f"Predicted Label: {predicted_label}")
        print(f"Confidence: {confidence:.2f}%")

    except FileNotFoundError:
        print(f"ERROR: Image file not found at path: {image_path}")
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(f"An unexpected error occurred during prediction: {e}")

In [80]:
train.sample(3)

0  label  \
2077  5\License_416\4404000000002947304207.jpg      4   
3032  7\License_607\4404000000002942756648.jpg      6   
3964  8\License_793\4404000000002943941939.jpg      7   

                                                path  
2077  image/5/License_416/4404000000002947304207.jpg  
3032  image/7/License_607/4404000000002942756648.jpg  
3964  image/8/License_793/4404000000002943941939.jpg

In [81]:
class_names = ["Toyota_Camry", "Toyota_Corolla", "Toyota_Corolla_EX", "Buick_LaCrosse", "Volkswagen_Magotan", "Audi_A4",
               "Nissan_Sylphy", "Nissan_Tiida", "Honda_Accord", "Ford_Focus"]
predict_single_image(trainer.model, "image/1/License_20/4404000000002940725419.jpg", transform, class_names)

tensor([[0.7763, 0.0257, 0.0138, 0.0145, 0.0852, 0.0027, 0.0174, 0.0154, 0.0448,
         0.0041]], device='cuda:0')
Predicted Class Index: 0
Predicted Label: Toyota_Camry
Confidence: 77.63%


In [82]:
class_names = ["Toyota_Camry", "Toyota_Corolla", "Toyota_Corolla_EX", "Buick_LaCrosse", "Volkswagen_Magotan", "Audi_A4",
               "Nissan_Sylphy", "Nissan_Tiida", "Honda_Accord", "Ford_Focus"]
predict_single_image(trainer.model, "image/6/License_531/4404000000002947607633.jpg", transform, class_names)

tensor([[0.0297, 0.0917, 0.0447, 0.0225, 0.0231, 0.6345, 0.0236, 0.0437, 0.0391,
         0.0473]], device='cuda:0')
Predicted Class Index: 5
Predicted Label: Audi_A4
Confidence: 63.45%
